


### Preparación de datos: Realizar la preparación de los datos teniendo en cnuenta las características de los algoritmos de agrupación a emplear.

In [39]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from sklearn.datasets import make_blobs



df = pd.read_csv("Datos_SenecaféAlpes.csv", sep=";")



df


ModuleNotFoundError: No module named 'sklearn.datasets.samples_generator'

In [34]:
print(df.columns)
print(df.shape)

Index(['ID', 'Area', 'Perimetro', 'LongitudEjeMayor', 'LongitudEjeMenor',
       'RelacionAspecto', 'Excentricidad', 'AreaConvexa',
       'DiametroEquivalente', 'Medida', 'Solidez', 'Redondez', 'Compacidad',
       'FactorForma1', 'FactorForma2', 'FactorForma3', 'FactorForma4',
       'DefectoVisible', 'MétodoSecado'],
      dtype='object')
(14291, 19)


In [35]:
# Tipos de datos y valores nulos
df.info()

# Estadísticas descriptivas de las variables numéricas
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14291 entries, 0 to 14290
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   14291 non-null  object 
 1   Area                 14291 non-null  int64  
 2   Perimetro            13054 non-null  float64
 3   LongitudEjeMayor     13890 non-null  float64
 4   LongitudEjeMenor     14291 non-null  float64
 5   RelacionAspecto      13825 non-null  object 
 6   Excentricidad        13687 non-null  float64
 7   AreaConvexa          12868 non-null  float64
 8   DiametroEquivalente  12368 non-null  float64
 9   Medida               14291 non-null  float64
 10  Solidez              11985 non-null  float64
 11  Redondez             12228 non-null  float64
 12  Compacidad           13641 non-null  float64
 13  FactorForma1         13172 non-null  float64
 14  FactorForma2         13185 non-null  float64
 15  FactorForma3         13813 non-null 

,Area,Perimetro,LongitudEjeMayor,LongitudEjeMenor,Excentricidad,AreaConvexa,DiametroEquivalente,Medida,Solidez,Redondez,Compacidad,FactorForma1,FactorForma2,FactorForma3,FactorForma4
count,14291.000000,1.305400e+04,13890.000000,14291.000000,13687.000000,12868.000000,12368.000000,14291.000000,11985.000000,12228.000000,13641.000000,13172.000000,13185.000000,13813.000000,13132.000000
mean,53055.408999,7.729870e+05,319.985592,202.178613,0.749977,53575.397809,253.001741,0.749844,0.986774,0.873080,0.799242,0.006560,0.001712,0.643183,0.994292
std,29396.080372,3.266493e+05,86.378452,45.494541,0.099438,29566.387814,60.542330,0.050774,0.025947,0.063237,0.067643,0.001164,0.000601,0.100857,0.039081
min,-62716.000000,-1.012143e+06,-421.444657,-200.838672,-0.835004,-78423.000000,-448.402605,-0.798706,-0.989042,-0.896861,-0.843901,-0.007982,-0.002673,-0.683269,-0.998527
25%,36338.000000,6.768608e+05,253.319858,175.881052,0.715144,36720.000000,215.302463,0.718767,0.985597,0.832824,0.762127,0.005903,0.001151,0.581047,0.993663
50%,44660.000000,7.720345e+05,296.682345,192.437870,0.764392,45107.500000,238.579492,0.760232,0.988279,0.883353,0.800994,0.006645,0.001691,0.641648,0.996377
75%,61311.000000,9.554098e+05,376.548109,216.847844,0.810441,62109.250000,279.672481,0.786942,0.989991,0.916803,0.834405,0.007273,0.002169,0.696366,0.997889
max,254616.000000,1.921685e+06,738.860154,460.198497,0.911423,251082.000000,569.374358,0.866195,0.994378,0.990685,0.987303,0.010451,0.003665,0.974767,0.999733


Lo primero a realizar será una limpieza basica de datos sobre nulos y duplicados. Ya que los algoritmos de clustering no funcionan bien si usamos datos nulos y puede crear ruido inncesario si usamos columnas duplicadas

In [36]:
# Valores nulos
df.isna().sum()

# Duplicados
df.duplicated().sum()

np.int64(461)

In [37]:
# --- Guardar cantidad antes ---
antes = df.shape[0]

# --- Eliminar granos con valores nulos o granos duplicados ---
df = df.dropna()
df = df.drop_duplicates()

# --- Guardar cantidad después ---
despues = df.shape[0]


print(f"Filas antes: {antes}")
print(f"Filas después: {despues}")
print(f"Se eliminaron {antes - despues} duplicados o nulos")




Filas antes: 14291
Filas después: 3495
Se eliminaron 10796 duplicados o nulos


Lo siguiente que se realizará será el one_hot_encoding para las variables categoricas, que identificamos 3, RelaciónAspecto: Indica si el grano es Alargado (>1.3) o Redondeado (≤1.3), DefectoVisible: Permite diferenciar entre granos sanos normales y granos con defectos visibles (Normal, Defectuoso) y MétodoSecado: Refleja una categoría relevante en el dominio cafetero, usada en caracterización de calidad y perfiles de taza (Natural, Lavado, Honey).

In [38]:
# --- 1. One-Hot Encoding para variables categóricas ---
df_encoded = pd.get_dummies(df, columns=["RelacionAspecto", "DefectoVisible", "MétodoSecado"], drop_first=True)

print("Shape final de dataframe:", df_encoded.shape)
df_encoded.head(5)

Shape final de dataframe: (3495, 25)


,ID,Area,Perimetro,LongitudEjeMayor,LongitudEjeMenor,Excentricidad,AreaConvexa,DiametroEquivalente,Medida,Solidez,...,FactorForma4,RelacionAspecto_Redondeado,RelacionAspecto_alargado,RelacionAspecto_redondeado,DefectoVisible_normal,MétodoSecado_Lavado,MétodoSecado_Natural,MétodoSecado_honey,MétodoSecado_lavado,MétodoSecado_natural
2,G007054,59965,994266.00,389.088529,197.967275,0.860886,60910.0,276.314692,0.661581,0.984485,...,0.991211,False,False,False,False,False,True,False,False,False
4,G013353,39324,737773.00,262.520242,191.176858,0.685326,39758.0,223.760747,0.775392,0.989084,...,0.997630,False,False,False,False,True,False,False,False,False
10,G001140,39948,731068.00,244.969135,207.964148,0.528489,40386.0,225.529096,0.772032,0.989155,...,0.998402,True,False,False,False,True,False,False,False,False
11,G007741,39392,746.64,265.915971,189.144664,0.702894,39836.0,223.954130,0.725291,0.988854,...,0.997194,False,False,False,False,False,True,False,False,False
19,G009423,47899,825844.00,317.342519,194.281893,0.790691,48932.0,246.955261,0.790569,0.978889,...,0.989181,False,False,False,True,False,False,False,True,False
